# Streaming Data

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import *

from operator import add
from pyspark.sql import functions as F

from functools import reduce
from pyspark.sql import DataFrame

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
conf = SparkConf().setAppName('learning-pyspark').setMaster('local[*]') 
sc = SparkContext.getOrCreate(conf)

spark = SparkSession(sc)

23/01/18 16:36:15 WARN Utils: Your hostname, Christoss-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.10.11 instead (on interface en0)
23/01/18 16:36:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/18 16:36:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#load the data

#defining the schema
df_schema = StructType([StructField("video_id", StringType(), True),
                       StructField("title", StringType(), True),
                       StructField("publishedAt", StringType(), True),
                       StructField("channelId", StringType(), True),
                       StructField("channelTitle", StringType(), True),
                       StructField("categoryId", IntegerType(), True),
                       StructField("trending_date", StringType(), True),
                       StructField("tags", StringType(), True),
                       StructField("view_count", IntegerType(), True),
                       StructField("likes", IntegerType(), True),
                       StructField("dislikes", IntegerType(), True),
                       StructField("comment_count", IntegerType(), True),
                       StructField("thumbnail_link", StringType(), True),
                       StructField("comments_disabled", BooleanType(), True),
                       StructField("ratings_disabled", BooleanType(), True),
                       StructField("description", StringType(), True)])

#load the datasets
df_br = spark.read.csv('Data/BR_youtube_trending_data.csv', schema = df_schema, header=True)
df_ca = spark.read.csv('Data/CA_youtube_trending_data.csv', schema = df_schema, header=True)
df_de = spark.read.csv('Data/DE_youtube_trending_data.csv', schema = df_schema, header=True)
df_fr = spark.read.csv('Data/FR_youtube_trending_data.csv', schema = df_schema, header=True)
df_gb = spark.read.csv('Data/GB_youtube_trending_data.csv', schema = df_schema, header=True)
df_in = spark.read.csv('Data/IN_youtube_trending_data.csv', schema = df_schema, header=True)
df_jp = spark.read.csv('Data/JP_youtube_trending_data.csv', schema = df_schema, header=True)
df_kr = spark.read.csv('Data/KR_youtube_trending_data.csv', schema = df_schema, header=True)
df_mx = spark.read.csv('Data/MX_youtube_trending_data.csv', schema = df_schema, header=True)
df_ru = spark.read.csv('Data/RU_youtube_trending_data.csv', schema = df_schema, header=True)
df_us = spark.read.csv('Data/US_youtube_trending_data.csv', schema = df_schema, header=True)

#defining the column country
df_br = df_br.withColumn("Country", F.lit('BR'))
df_ca = df_ca.withColumn("Country", F.lit('CA'))
df_de = df_de.withColumn("Country", F.lit('DE'))
df_fr = df_fr.withColumn("Country", F.lit('DE'))
df_gb = df_gb.withColumn("Country", F.lit('GB'))
df_in = df_in.withColumn("Country", F.lit('IN'))
df_jp = df_jp.withColumn("Country", F.lit('JP'))
df_kr = df_kr.withColumn("Country", F.lit('KR'))
df_mx = df_de.withColumn("Country", F.lit('MX'))
df_ru = df_ru.withColumn("Country", F.lit('RU'))
df_us = df_us.withColumn("Country", F.lit('US'))

# create merged dataframe
df_list=[df_br,df_ca,df_de,df_fr,df_gb,df_in,df_jp,df_kr,df_mx,df_ru,df_us]
df_final = reduce(DataFrame.unionAll, df_list)

In [4]:
df_fr.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- channelId: string (nullable = true)
 |-- channelTitle: string (nullable = true)
 |-- categoryId: integer (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- view_count: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- Country: string (nullable = false)



I will work only with a subset of the data and I have randomly selected the videos from France

In [5]:
#change the types

df_fr = df_fr.withColumn("publishedAt",to_timestamp("publishedAt"))
df_fr = df_fr.withColumn("trending_date",to_timestamp("trending_date"))

In [6]:
df_fr = df_fr.withColumn("published_date", F.to_date("publishedAt"))

In [7]:
df_fr = df_fr.sort('published_date')

In [8]:
#removing na values from the column 'published_date'

df_fr = df_fr.dropna(subset=['published_date'])

In [9]:
#drop duplicates

df_fr = df_fr.drop_duplicates(subset=['video_id'])

In [10]:
#Count of Null of 'published_date' Column
from pyspark.sql.functions import col,isnan, when, count
df_fr.select([count(when(col('published_date').isNull(), 'published_date'))\
              .alias('published_date')]
            ).show()


+--------------+
|published_date|
+--------------+
|             0|
+--------------+



In [11]:
days = df_fr.select('published_date').distinct().collect()

#sort the days
days_sorted = sorted(days)

In [12]:
#how many days we have
len(days_sorted)

851

In [14]:
for day in days_sorted[:300]:
    df_day = df_fr.where(col('published_date') == day[0])
    df_day.coalesce(1).write.mode('append').option('header','true').csv('Data/Data_Stream')

In [15]:
#defining the updated schema

df_schema = StructType([StructField("video_id", StringType(), True),
                       StructField("title", StringType(), True),
                       StructField("publishedAt", TimestampType(), True),
                       StructField("channelId", StringType(), True),
                       StructField("channelTitle", StringType(), True),
                       StructField("categoryId", IntegerType(), True),
                       StructField("trending_date", StringType(), True),
                       StructField("tags", StringType(), True),
                       StructField("view_count", IntegerType(), True),
                       StructField("likes", IntegerType(), True),
                       StructField("dislikes", IntegerType(), True),
                       StructField("comment_count", IntegerType(), True),
                       StructField("thumbnail_link", StringType(), True),
                       StructField("comments_disabled", BooleanType(), True),
                       StructField("ratings_disabled", BooleanType(), True),
                       StructField("description", StringType(), True),
                       StructField("published_date", DateType(), True)])

In [16]:
streaming = (spark.readStream.schema(df_schema)
            .option('maxFilesPerTrigger',1).csv('Data/Data_Stream/'))

In [17]:
#defining a column hour

streaming_hour = streaming.withColumn("published_hour", F.hour("publishedAt"))

In [18]:
#groupBy published hour

hour_count = streaming_hour.groupBy('published_hour').count().orderBy(F.desc('count'))

In [22]:
activityQuery = (hour_count.writeStream.queryName('hour_count').format('memory').outputMode('complete').start())

import time

for x in range(100):
    df_tempt = spark.sql('SELECT * FROM hour_count')
    if df_tempt.count()>0:
        df_tempt = df_tempt.dropna(subset=['published_hour'])
        df_tempt.show(24)
    time.sleep(1)

23/01/18 23:42:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/5y/vl62knx9149_y05z_s54ntph0000gn/T/temporary-13514ed1-5d4d-4cc1-a7b6-341bc884c22c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/18 23:42:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+--------------+-----+
|published_hour|count|
+--------------+-----+
|             0|    1|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            17|    1|
|             0|    1|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|    4|
|            17|    1|
|             8|    1|
|            23|    1|
|             0|    1|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|   13|
|            17|    3|
|            18|    3|
|            12|    2|
|            22|    1|
|             1|    1|
|            16|    1|
|             9|    1|
|             8|    1|
|            23|    1|
|             7|    1|
|            10|    1|
|             0|    1|
+--------------+-----+



...

+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|  689|
|            18|  480|
|            13|  220|
|            12|  207|
|            17|  207|
|            20|  175|
|            11|  174|
|            21|  171|
|            15|  120|
|            14|  110|
|            16|  107|
|            22|  103|
|             1|   97|
|             7|   96|
|             0|   90|
|            10|   77|
|            23|   68|
|             9|   42|
|             2|   33|
|             3|   29|
|             6|   28|
|             8|   22|
|             4|   15|
|             5|   10|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|  700|
|            18|  489|
|            13|  227|
|            12|  207|
|            17|  207|
|            20|  178|
|            11|  175|
|            21|  174|
|            15|  121|
|            14|  111|
|            16|  107|
|            22|  106|
|             1|   99|
|             7|   96|
|             0|   90|
|            10|   78|
|            23|   68|
|             9|   42|
|             2|   34|
|             3|   29|
|             6|   28|
|             8|   23|
|             4|   15|
|             5|   10|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|  716|
|            18|  500|
|            13|  233|
|            17|  209|
|            12|  208|
|            20|  180|
|            11|  180|
|            21|  178|
|            15|  122|
|            14|  112|
|            16|  109|
|            22|  106|
|             1|  103|
|             7|   99|
|             0|   91|
|            10|   79|
|            23|   68|
|             9|   44|
|             2|   35|
|             6|   29|
|             3|   29|
|             8|   24|
|             4|   15|
|             5|   11|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|  719|
|            18|  505|
|            13|  234|
|            17|  211|
|            12|  210|
|            11|  182|
|            20|  181|
|            21|  180|
|            15|  123|
|            14|  115|
|            16|  111|
|            22|  107|
|             1|  103|
|             7|   99|
|             0|   91|
|            10|   80|
|            23|   68|
|             9|   44|
|             2|   35|
|             6|   29|
|             3|   29|
|             8|   24|
|             4|   15|
|             5|   11|
+--------------+-----+



+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|  727|
|            18|  509|
|            13|  238|
|            12|  218|
|            17|  213|
|            11|  189|
|            20|  182|
|            21|  180|
|            15|  125|
|            14|  118|
|            16|  114|
|            22|  109|
|             1|  105|
|             7|   99|
|             0|   93|
|            10|   84|
|            23|   70|
|             9|   45|
|             2|   36|
|             3|   30|
|             6|   29|
|             8|   24|
|             4|   15|
|             5|   11|
+--------------+-----+



In [20]:
activityQuery.stop()

23/01/18 20:02:01 WARN Shell: Interrupted while joining on: Thread[Thread-277235,5,]
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1257)
	at java.lang.Thread.join(Thread.java:1331)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1042)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1002)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.FileUtil.readLink(FileUtil.java:211)
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileLinkStatusInternal(RawLocalFileSystem.java:1113)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1102)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatus(RawLocalFileSystem.java:1073)
	at org.apach

In [21]:
df_tempt.show()

+--------------+-----+
|published_hour|count|
+--------------+-----+
|            19|  547|
|            18|  385|
|            13|  188|
|            17|  168|
|            12|  157|
|            11|  149|
|            20|  148|
|            21|  137|
|            15|  101|
|            14|   91|
|            16|   89|
|             7|   89|
|            22|   85|
|             0|   78|
|             1|   74|
|            10|   66|
|            23|   63|
|             9|   36|
|             2|   26|
|             3|   25|
+--------------+-----+
only showing top 20 rows

